DONE
- find_branching_site
- validate_protein_keys
- check_branching_sites
- check_branching_sequences
- check_children_format
- check_branching_site_sequence_match



do the following;
- validate_branching_sites

TESTING_FUNCTION (function to build recursive tests for) = validate_branching_sites

use function called inject_branching_sites

def inject_branching_sites(branches, target_chain_number, new_branching_sites, current_chain_number=1):
    """
    Recursively injects or replaces the branching_sites list of a specific ubiquitin node.

    Args:
        branches (list): Top-level list of branching_sites.
        target_chain_number (int): The target chain number for injection.
        new_branching_sites (list): The new branching_sites to set.
        current_chain_number (int): Current depth of recursion.
    """
    for site in branches:
        child = site.get("children")
        if isinstance(child, dict):
            if child.get("chain_number") == target_chain_number:
                child["branching_sites"] = new_branching_sites
                return
            inject_branching_sites(child.get("branching_sites", []), target_chain_number, new_branching_sites, current_chain_number + 1)

Within the codebase of ubiquitinformatics in vscode, the existing tests for TESTING_FUNCTION etc. primarily tests this function in isolation or at the top level of the input dictionary. 

@pytest.mark.parametrize("ubiquitin_dict, should_raise, expected_parts", [

    # ✅ Test 1: Perfect structure
    ({
        "chain_number": 1,
        "branching_sites": [
            {"site_name": "M1", "sequence_id": "(M)QIF", "children": ""},
            {"site_name": "K6", "sequence_id": "IFV(K)TLT", "children": ""},
            {"site_name": "K11", "sequence_id": "LTG(K)TIT", "children": ""},
            {"site_name": "K27", "sequence_id": "ENV(K)AKI", "children": ""},
            {"site_name": "K29", "sequence_id": "VKA(K)IQD", "children": ""},
            {"site_name": "K33", "sequence_id": "IQD(K)EGI", "children": ""},
            {"site_name": "K48", "sequence_id": "FAG(K)QLE", "children": ""},
            {"site_name": "K63", "sequence_id": "NIQ(K)EST", "children": ""}
        ]
    }, False, None),

    # ❌ Test 2: Incorrect sequence_id mapping for one site
    ({
        "chain_number": 1,
        "branching_sites": [
            {"site_name": "M1", "sequence_id": "(M)QIF", "children": ""},
            {"site_name": "K6", "sequence_id": "IFV(K)TLT", "children": ""},
            {"site_name": "K11", "sequence_id": "LTG(K)TIT", "children": ""},
            {"site_name": "K27", "sequence_id": "ENV(K)AKI", "children": ""},
            {"site_name": "K29", "sequence_id": "VKA(K)IQD", "children": ""},
            {"site_name": "K33", "sequence_id": "IQD(K)EGI", "children": ""},
            {"site_name": "K48", "sequence_id": "NIQ(K)EST", "children": ""},
            {"site_name": "K63", "sequence_id": "FAG(K)QLE", "children": ""}
        ]
    }, True, [
        "does not correspond with the sequence_id",
        "NIQ(K)EST",
        "K63",
        "does not correspond with the sequence_id",
        "FAG(K)QLE"]), #raises validate_branching_sites error

    # ❌ Test 3: One invalid child format (not string or dict)
    ({
        "chain_number": 1,
        "branching_sites": [
            {"site_name": "M1", "sequence_id": "(M)QIF", "children": ""},
            {"site_name": "K6", "sequence_id": "IFV(K)TLT", "children": 42},
            {"site_name": "K11", "sequence_id": "LTG(K)TIT", "children": ""},
            {"site_name": "K27", "sequence_id": "ENV(K)AKI", "children": ""},
            {"site_name": "K29", "sequence_id": "VKA(K)IQD", "children": ""},
            {"site_name": "K33", "sequence_id": "IQD(K)EGI", "children": ""},
            {"site_name": "K48", "sequence_id": "NIQ(K)EST", "children": ""},
            {"site_name": "K63", "sequence_id": "FAG(K)QLE", "children": ""}
        ]
    }, True, [
        "Invalid children format:",
        "42",
        "K48",
        "does not correspond with the sequence_id",
        "NIQ(K)EST",
        "K63",
        "does not correspond with the sequence_id",
        "FAG(K)QLE"]),

    # ❌ Test 3a: One invalid child format (wrong string)
    ({
        "chain_number": 1,
        "branching_sites": [
            {"site_name": "M1", "sequence_id": "(M)QIF", "children": ""},
            {"site_name": "K6", "sequence_id": "IFV(K)TLT", "children": "42"},
            {"site_name": "K11", "sequence_id": "LTG(K)TIT", "children": ""},
            {"site_name": "K27", "sequence_id": "ENV(K)AKI", "children": ""},
            {"site_name": "K29", "sequence_id": "VKA(K)IQD", "children": ""},
            {"site_name": "K33", "sequence_id": "IQD(K)EGI", "children": ""},
            {"site_name": "K48", "sequence_id": "NIQ(K)EST", "children": ""},
            {"site_name": "K63", "sequence_id": "FAG(K)QLE", "children": ""}
        ]
    }, True, [
        "Invalid children format:",
        "42",
        "K48",
        "does not correspond with the sequence_id",
        "NIQ(K)EST",
        "K63",
        "does not correspond with the sequence_id",
        "FAG(K)QLE"]),

    # ❌ Test 4: Invalid site_name (not expected)
    ({
        "chain_number": 1,
        "branching_sites": [
            {"site_name": "M1", "sequence_id": "(M)QIF", "children": ""},
            {"site_name": "K99", "sequence_id": "IFV(K)TLT", "children": ""}, # Invalid site_name (not expected)
            {"site_name": "K11", "sequence_id": "LTG(K)TIT", "children": ""},
            {"site_name": "K27", "sequence_id": "ENV(K)AKI", "children": ""},
            {"site_name": "K29", "sequence_id": "VKA(K)IQD", "children": ""},
            {"site_name": "K33", "sequence_id": "IQD(K)EGI", "children": ""},
            {"site_name": "K48", "sequence_id": "NIQ(K)EST", "children": ""},
            {"site_name": "K63", "sequence_id": "FAG(K)QLE", "children": ""}
        ]
    }, True, ["Invalid sites found",
              "K99",
              "Ubiquitin 1",
              "Allowed sites",
              "M1",
              "K6",
              "K11",
              "K27",
              "K29",
              "K33",
              "K48",
              "K63"]),

    # ❌ Test 5: Missing one required site
    ({
        "chain_number": 1,
        "branching_sites": [
            {"site_name": "M1", "sequence_id": "(M)QIF", "children": ""},
            {"site_name": "K6", "sequence_id": "IFV(K)TLT", "children": ""}, # Invalid site_name (not expected)
            {"site_name": "K11", "sequence_id": "LTG(K)TIT", "children": ""},
            {"site_name": "K27", "sequence_id": "ENV(K)AKI", "children": ""}
            # Missing the rest
        ]
    }, True, ["Missing required sites",
              "K29",
              "K33",
              "K48",
              "K63"]),

    # ❌ Test 6: Invalid sequence_id name
    ({
        "chain_number": 1,
        "branching_sites": [
            {"site_name": "M1", "sequence_id": "(M)QIF", "children": ""},
            {"site_name": "K6", "sequence_id": "LTG(K)TIT", "children": ""},
            {"site_name": "K11", "sequence_id": "IFV(K)TLT", "children": ""}, 
            {"site_name": "K27", "sequence_id": "ENV(K)AKI", "children": ""},
            {"site_name": "K29", "sequence_id": "VKA(K)IQD", "children": ""},
            {"site_name": "K33", "sequence_id": "IQD(K)EGI", "children": ""},
            {"site_name": "K48", "sequence_id": "FAG(K)QLE", "children": ""},
            {"site_name": "K63", "sequence_id": "NIQ(K)EST", "children": ""}
        ]
    }, True, ["site_name:",
              "K6",
              "does not correspond with the sequence_id",
              "site_name:", 
              "K11", 
              "does not correspond with the sequence_id"]),

    # ❌ Test 7: SMAC with invalid site/sequence
    ({
        "chain_number": 1,
        "branching_sites": [
            {"site_name": "M1", "sequence_id": "(M)QIF", "children": ""},
            {"site_name": "K6", "sequence_id": "LTG(K)TIT", "children": ""},
            {"site_name": "K11", "sequence_id": "IFV(K)TLT", "children": "SMAC"}, 
            {"site_name": "K27", "sequence_id": "ENV(K)AKI", "children": ""},
            {"site_name": "K29", "sequence_id": "VKA(K)IQD", "children": ""},
            {"site_name": "K33", "sequence_id": "IQD(K)EGI", "children": ""},
            {"site_name": "K48", "sequence_id": "FAG(K)QLE", "children": ""},
            {"site_name": "K63", "sequence_id": "NIQ(K)EST", "children": ""}
        ]
    }, True, ["site_name:",
              "K6",
              "does not correspond with the sequence_id",
              "site_name:", 
              "K11", 
              "does not correspond with the sequence_id"]),

    # ❌ Test 8: Children as a list (invalid)
    ({
        "chain_number": 1,
        "branching_sites": [
            {"site_name": "M1", "sequence_id": "(M)QIF", "children": ""},
            {"site_name": "K6", "sequence_id": "IFV(K)TLT", "children": ""},
            {"site_name": "K11", "sequence_id": "LTG(K)TIT", "children": ""}, 
            {"site_name": "K27", "sequence_id": "ENV(K)AKI", "children": ""},
            {"site_name": "K29", "sequence_id": "VKA(K)IQD", "children": ["wrong"]},
            {"site_name": "K33", "sequence_id": "IQD(K)EGI", "children": ""},
            {"site_name": "K48", "sequence_id": "FAG(K)QLE", "children": ""},
            {"site_name": "K63", "sequence_id": "NIQ(K)EST", "children": ""}
        ]
    }, True, ["Invalid children format", 
              "wrong"]),

    # ❌ Test 9: Multiple errors (bad sequence + bad child format)
    ({
        "chain_number": 1,
        "branching_sites": [
            {"site_name": "M1", "sequence_id": "(M)QIF", "children": ""},
            {"site_name": "K6", "sequence_id": "LTG(K)TIT", "children": ""},
            {"site_name": "K11", "sequence_id": "IFV(K)TLT", "children": ""}, 
            {"site_name": "K27", "sequence_id": "ENV(K)AKI", "children": ""},
            {"site_name": "K29", "sequence_id": "VKA(K)IQD", "children": ["wrong"]},
            {"site_name": "K33", "sequence_id": "IQD(K)EGI", "children": ""},
            {"site_name": "K48", "sequence_id": "FAG(K)QLE", "children": ""},
            {"site_name": "K63", "sequence_id": "NIQ(K)EST", "children": ""}
        ]
    }, True, ["does not correspond with the sequence_id", 
              "Invalid children format"]),

    # ❌ Test 10: Correct branching sites but missing one sequence_id
    ({
        "chain_number": 1,
        "branching_sites": [
            {"site_name": "M1", "sequence_id": "(M)QIF", "children": ""},
            {"site_name": "K6", "sequence_id": "IFV(K)TLT", "children": ""},
            {"site_name": "K11", "sequence_id": "LTG(K)TIT", "children": ""},
            {"site_name": "K27", "sequence_id": "ENV(K)AKI", "children": ""},
            {"site_name": "K29", "sequence_id": "VKA(K)IQD", "children": ""},
            {"site_name": "K33", "sequence_id": "IQD(K)EGI", "children": ""},
            {"site_name": "K48", "sequence_id": "FAG(K)QLE", "children": ""}
            # K63 missing
        ]
    }, True, ["Missing required sites", 
              "Allowed sites", 
              "Missing required sequences", 
              "Allowed sequences"]
)
])
def test_validate_branching_sites(ubiquitin_dict, should_raise, expected_parts):

    if should_raise:
        with pytest.raises(AssertionError) as exc_info:
            validate_branching_sites(ubiquitin_dict)
        
        error_msg = str(exc_info.value)
        assert match_assertion_error_contains(error_msg, expected_parts), \
            f"Expected parts {expected_parts} not found in error: {error_msg}"
    
    else:
        validate_branching_sites(ubiquitin_dict)  # Should not raise

The goal now is to ensure these validations are triggered correctly for the nested dictionaries (with 4 of more ubiquitins bound together) during the recursive traversal performed by iterate_through_ubiquitin and its helper inner_wrapper_iterate_through_ubiquitin. Please write a new test for each test that are triggered for these nested dictionaries. Use the base nested dictionary found in test_data.py. Make sure the tests use iterate_through_ubiquitin as the function. Do not apply the code within vscode